Reference
https://www.jstor.org/stable/2685594  
https://github.com/desaizeeshan22/OLS-with-Newey-West-Correction/blob/master/PS2Q2.py  
https://github.com/aamanpandey/Newey-West-estimator/blob/master/Newey-West%20estimator_Code.txt  
https://zhuanlan.zhihu.com/p/54913149

In [598]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)
data=pd.read_csv("data.csv")

In [599]:
data

,y,x1,x2
0,53.00,2.0,2.03372
1,62.00,0.5,3.69980
2,97.00,9.0,4.57720
3,63.30,3.4,2.75830
4,5.20,9.5,6.23000
5,54.30,2.6,83.60000
6,12.80,6.3,50.20000
7,63.50,2.0,20.50000
8,30.44,4.5,4.75000
9,50.00,9.7,9.55000


In [600]:
#ols with normal se
reg=smf.ols('y ~ 1 + x1 + x2',data=data).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                   0.06960
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.933
Time:                        22:13:07   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927     20.511      2.852      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [601]:
#ols matrix form
x=sm.add_constant(data.iloc[:,1:].values)
y=data.iloc[:,0].values
b=np.linalg.solve(x.T@x,x.T@y)#coefficient
e=y-x@b#residual
mse=0
for i in e**2:
    mse+=i
mse=mse/(x.shape[0]-x.shape[1])
vb=mse*np.linalg.inv(x.T@x)
#vb = e.var()*np.linalg.inv(x.T@x) #function var>n, not n-k-1
se = np.sqrt(np.diagonal(vb))
t = b/se
print(se)

[20.51075028  2.95090377  0.39378159]


In [602]:
#HC0 robust se
reg = smf.ols('y ~ 1 + x1 + x2',data=data).fit(cov_type='HC0')
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                    0.2056
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.818
Time:                        22:13:23   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927      7.970      7.339      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [603]:
#HC0 matrix form
part2b_het=np.nan
part1=np.linalg.inv(x.T @ x)
part3=np.linalg.inv(x.T @ x)
e_square=e**2
part2a=x.T
part2b_het=np.eye(x.shape[0])
part2c=x
#對角=單個殘差 不需要除n-k-1
for i in range(0,len(e)):
    part2b_het[i]*=e_square[i]
hc0=part1@(part2a@part2b_het@part2c)@part3
print(hc0[0][0]**0.5)
print(hc0[1][1]**0.5)
print(hc0[2][2]**0.5)

7.970106846449044
2.5076309922304434
0.23395636232570802


In [604]:
#HC1 robust se
reg = smf.ols('y ~ 1 + x1 + x2',data=data).fit(cov_type='HC1')
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                    0.1495
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.863
Time:                        22:13:40   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927      9.346      6.259      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [605]:
#HC1 robust se
part2b_het=np.nan
part1=np.linalg.inv(x.T @ x)
part3=np.linalg.inv(x.T @ x)
e_square=e**2
part2a=x.T
part2b_het=np.eye(x.shape[0])
part2c=x

for i in range(0,len(e)):
    part2b_het[i]*=e_square[i]
hc0=part1@(part2a@part2b_het@part2c)@part3
hc1=(x.shape[0]/(x.shape[0]-x.shape[1]))*hc0
print(hc1[0][0]**0.5)
print(hc1[1][1]**0.5)
print(hc1[2][2]**0.5)

9.345778690015303
2.940457981444834
0.27433815224086533


In [606]:
#HC2 robust se
reg = smf.ols('y ~ 1 + x1 + x2',data=data).fit(cov_type='HC2')
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                    0.1346
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.876
Time:                        22:14:18   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:                  HC2                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927      8.907      6.567      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [545]:
#HC2 robust se
part2b_het=np.nan
part1=np.linalg.inv(x.T @ x)
part3=np.linalg.inv(x.T @ x)
e_square=e**2
part2a=x.T
part2b_het=np.eye(x.shape[0])
part2c=x

for i in range(0,len(e)):
    hii=(np.reshape(x[i,:],(1,x.shape[1]))@part1@np.reshape(x.T[:,i],(x.shape[1],1)))
    adjust_e=(e_square[i]/(1-hii))[0][0]
    part2b_het[i]*=adjust_e
hc2=part1@(part2a@part2b_het@part2c)@part3
print(hc2[0][0]**0.5)
print(hc2[1][1]**0.5)
print(hc2[2][2]**0.5)

8.9066734469543
2.869889038259061
0.29683781079848015


In [607]:
#HC3 robust se
reg = smf.ols('y ~ 1 + x1 + x2',data=data).fit(cov_type='HC3')
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                   0.07578
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.928
Time:                        22:14:39   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927      9.987      5.857      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [608]:
#HC3 robust se
part2b_het=np.nan
part1=np.linalg.inv(x.T @ x)
part3=np.linalg.inv(x.T @ x)
e_square=e**2
part2a=x.T
part2b_het=np.eye(x.shape[0])
part2c=x

for i in range(0,len(e)):
    hii=(np.reshape(x[i,:],(1,x.shape[1]))@part1@np.reshape(x.T[:,i],(x.shape[1],1)))
    adjust_e=(e_square[i]/(1-hii)**2)[0][0]
    part2b_het[i]*=adjust_e
hc2=part1@(part2a@part2b_het@part2c)@part3
print(hc2[0][0]**0.5)
print(hc2[1][1]**0.5)
print(hc2[2][2]**0.5)

9.987359361003705
3.300815345253009
0.4209443524111706


White:part2b對角元素不用mse，轉用各殘差的平方(HC0)
Newey and West:公式前半針對殘差異質 後半針對殘差相關(HAC)

In [609]:
#HAC robust se
reg = smf.ols('y ~ 1 + x1 + x2',data=data).fit(cov_type='HAC',cov_kwds={'maxlags':1})
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.229
Method:                 Least Squares   F-statistic:                    0.2588
Date:                Fri, 14 Jan 2022   Prob (F-statistic):              0.778
Time:                        22:15:53   Log-Likelihood:                -51.937
No. Observations:                  11   AIC:                             109.9
Df Residuals:                       8   BIC:                             111.1
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     58.4927      7.413      7.890      0.0

C:\Users\antho\anaconda3\envs\py38_jq\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=11
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [610]:
w=0.5
part2b_het=np.nan
part1=np.linalg.inv(x.T @ x)
part3=np.linalg.inv(x.T @ x)
e_square=e**2
part2a=x.T
part2b_het=np.eye(x.shape[0])
part2c=x

#het
for i in range(0,len(e)):
    part2b_het[i]*=e_square[i]

#ar    
for j in range(1,len(e)):
    if j==1:
        ar_p1=w*e[j]*e[j-1]
        ar_p2=np.reshape(x[j,:],(x.shape[1],1))@np.reshape(x[j-1,:],(1,x.shape[1]))
        ar_p3=np.reshape(x[j-1,:],(x.shape[1],1))@np.reshape(x[j,:],(1,x.shape[1]))
        ar=ar_p1*(ar_p2+ar_p3)
    else:
        ar_p1=w*e[j]*e[j-1]
        ar_p2=np.reshape(x[j,:],(x.shape[1],1))@np.reshape(x[j-1,:],(1,x.shape[1]))
        ar_p3=np.reshape(x[j-1,:],(x.shape[1],1))@np.reshape(x[j,:],(1,x.shape[1]))
        ar+=(ar_p1*(ar_p2+ar_p3))
    
hac=part1@((part2a@part2b_het@part2c)+ar)@part3
print(test[0][0]**0.5) 
print(test[1][1]**0.5) 
print(test[2][2]**0.5) 

7.413482712502961
2.50348967865012
0.2188026408605374
